# Kubeflow Fairing Introduction

Kubeflow Fairing is a Python package that streamlines the process of `building`, `training`, and `deploying` machine learning (ML) models in a hybrid cloud environment. By using Kubeflow Fairing and adding a few lines of code, you can run your ML training job locally or in the cloud, directly from Python code or a Jupyter notebook. After your training job is complete, you can use Kubeflow Fairing to deploy your trained model as a prediction endpoint.


# How does Kubeflow Fairing work

Kubeflow Fairing 
1. Packages your Jupyter notebook, Python function, or Python file as a Docker image
2. Deploys and runs the training job on Kubeflow or AI Platform. 
3. Deploy your trained model as a prediction endpoint on Kubeflow after your training job is complete.


# Goals of Kubeflow Fairing project

- Easily package ML training jobs: Enable ML practitioners to easily package their ML model training code, and their code’s dependencies, as a Docker image.
- Easily train ML models in a hybrid cloud environment: Provide a high-level API for training ML models to make it easy to run training jobs in the cloud, without needing to understand the underlying infrastructure.
- Streamline the process of deploying a trained model: Make it easy for ML practitioners to deploy trained ML models to a hybrid cloud environment.


> Note: Before fairing workshop, please read `README.md` under `02_01_fairing_introduction`


In [8]:
# check fairing is installed 
!pip show kubeflow-fairing

Name: kubeflow-fairing
Version: 1.0.2
Summary: Kubeflow Fairing Python SDK.
Home-page: https://github.com/kubeflow/fairing
Author: Kubeflow Authors
Author-email: hejinchi@cn.ibm.com
License: Apache License Version 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: google-cloud-storage, docker, google-cloud-logging, oauth2client, numpy, nbconvert, retrying, kubernetes, kubeflow-pytorchjob, six, kubeflow-tfjob, grpcio, azure-mgmt-storage, tornado, future, azure-storage-file, boto3, python-dateutil, setuptools, requests, httplib2, kfserving, urllib3, ibm-cos-sdk, notebook, google-auth, google-api-python-client, cloudpickle
Required-by: 


## Basic Example

If you see any issues, please restart notebook. It's probably because of new installed packages.

Click `Kernel` -> `Restart & Clear Output`

In [7]:
#%%writefile train_model.py
import os
import sys
import tensorflow as tf
import numpy as np

def nkTrain():
    # Genrating random linear data 
    # There will be 50 data points ranging from 0 to 50 
    x = np.linspace(0, 50, 50) 
    y = np.linspace(0, 50, 50) 

    # Adding noise to the random linear data 
    x += np.random.uniform(-4, 4, 50) 
    y += np.random.uniform(-4, 4, 50) 

    n = len(x) # Number of data points 

    X = tf.placeholder("float") 
    Y = tf.placeholder("float")
    W = tf.Variable(np.random.randn(), name = "W") 
    b = tf.Variable(np.random.randn(), name = "b") 
    learning_rate = 0.01
    training_epochs = 1000
    
    # Hypothesis 
    y_pred = tf.add(tf.multiply(X, W), b) 

    # Mean Squared Error Cost Function 
    cost = tf.reduce_sum(tf.pow(y_pred-Y, 2)) / (2 * n)

    # Gradient Descent Optimizer 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost) 

    # Global Variables Initializer 
    init = tf.global_variables_initializer() 


    sess = tf.Session()
    sess.run(init) 
      
    # Iterating through all the epochs 
    for epoch in range(training_epochs): 
          
        # Feeding each data point into the optimizer using Feed Dictionary 
        for (_x, _y) in zip(x, y): 
            sess.run(optimizer, feed_dict = {X : _x, Y : _y}) 
          
        # Displaying the result after every 50 epochs 
        if (epoch + 1) % 50 == 0: 
            # Calculating the cost a every epoch 
            c = sess.run(cost, feed_dict = {X : x, Y : y}) 
            print("Epoch", (epoch + 1), ": cost =", c, "W =", sess.run(W), "b =", sess.run(b)) 
      
    # Storing necessary values to be used outside the Session 
    training_cost = sess.run(cost, feed_dict ={X: x, Y: y}) 
    weight = sess.run(W) 
    bias = sess.run(b) 

    print('Weight: ', weight, 'Bias: ', bias)

## Local training for development



In [3]:
nkTrain()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 50 : cost = 6.010859 W = 1.0002372 b = 0.260292
Epoch 100 : cost = 5.8873587 W = 0.99437755 b = 0.5443702
Epoch 150 : cost = 5.800606 W = 0.9891751 b = 0.7965846
Epoch 200 : cost = 5.741632 W = 0.9845562 b = 1.0205089
Epoch 250 : cost = 5.703499 W = 0.9804554 b = 1.2193136
Epoch 300 : cost = 5.680856 W = 0.9768146 b = 1.3958187
Epoch 350 : cost = 5.6695924 W = 0.9735821 b = 1.5525293
Epoch 400 : cost = 5.6665587 W = 0.97071224 b = 1.6916598
Epoch 450 : cost = 5.669358 W = 0.9681642 b = 1.8151871
Epoch 500 : cost = 5.676172 W = 0.9659021 b = 1.9248542
Epoch 550 : cost = 5.685632 W = 0.96389365 b = 2.0222213
Epoch 600 : cost = 5.6967263 W = 0.96211064 b = 2.1086648
Epoch 650 : cost = 5.708691 W = 0.9605275 b = 2.1854148
Epoch 700 : cost = 5.720987 W = 0.9591218 b = 2.2535608
Epoch 750 : cost = 5.7332225 W = 0.95787376 b = 2.314065
Epoch 800 : cost = 5.745124 W = 0.95676595 b = 2.3677742
Ep

## Remote training

We will show you how to remotely run training job in kubernetes cluster. You can use `ECR` as your container image registry.

In [11]:
!nkode create:image

hello auto from ./src/commands/hello.js
sh /usr/lib/node_modules/nkode/scripts/remoteTrain/remote_train.sh  auto data tensorflow/tensorflow:1.15.0-py3
auto data tensorflow/tensorflow:1.15.0-py3
[I 201022 20:33:44 utils:320] IMDS ENDPOINT: http://169.254.169.254/
[W 201022 20:33:44 function:49] The FunctionPreProcessor is optimized for using in a notebook or IPython environment. For it to work, the python version should be same for both local python and the python in the docker. Please look at alternatives like BasePreprocessor or FullNotebookPreprocessor.
[W 201022 20:33:44 tasks:62] Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
[I 201022 20:33:44 tasks:66] Building the docker image.
[I 201022 20:33:44 cluster:46] Building image using cluster builder.
[W 201022 20:33:44 base:94] /usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 201022 20:33:44 base:107] Creating docker context: /tmp/

  Created wheel for tornado: filename=tornado-6.0.4-cp36-cp36m-linux_x86_64.whl size=429305 sha256=a532e6d8826208afbffa2da4656bf31aa5286c09f03aa132e226832134d90ce9
  Stored in directory: /tmp/pip-ephem-wheel-cache-8b_htg7x/wheels/93/84/2f/409c7b2bb3afc3aa727f7ee8787975e0793f74d1165f4d0104
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=493275 sha256=5deb686a099590b706882d658ea640b2cdadef98af28c1284fc6db04db55aabd
  Stored in directory: /tmp/pip-ephem-wheel-cache-8b_htg7x/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.7.0-py2.py3-none-any.whl size=71964 sha256=ccb7e26ae27ffb4ca2bafd62a2ab1e7a3c0230a48584d429007db86ad4079f14
  Stored in directory: /tmp/pip-ephem-wheel-cache-8b_htg7x/wheels/a7/10/5b/5d6543382dc5cc86022e490619c8db1253c22d50a57493262b


[W 201022 20:35:57 job:101] The job fairing-job-kqw78 launched.
[W 201022 20:35:57 manager:298] Waiting for fairing-job-kqw78-kc72q to start...
[W 201022 20:35:57 manager:298] Waiting for fairing-job-kqw78-kc72q to start...
[W 201022 20:35:58 manager:298] Waiting for fairing-job-kqw78-kc72q to start...
[I 201022 20:36:41 manager:304] Pod started running True

  Created wheel for retrying: filename=retrying-1.3.3-cp36-none-any.whl size=9533 sha256=1cf97f0c1dd8f029209fc294c9ec408ae115d2ebb0deb2bd19124a6df55df4ae
  Stored in directory: /tmp/pip-ephem-wheel-cache-8b_htg7x/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
  Created wheel for pandocfilters: filename=pandocfilters-1.4.2-cp36-none-any.whl size=10338 sha256=e3ebc25d79a86842419ea9c2c084dc34aedd1be3f202a665bedafd72181f6c74
  Stored in directory: /tmp/pip-ephem-wheel-cache-8b_htg7x/wheels/39/01/56/f1b08a6275acc59e846fa4c1e1b65dbc1919f20157d9e66c20
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.7

2020-10-22 20:36:41.940404: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)[W 201022 20:36:57 job:173] Cleaning up job fairing-job-kqw78...

2020-10-22 20:36:41.940427: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fairing-job-kqw78-kc72q): /proc/driver/nvidia/version does not exist
2020-10-22 20:36:41.940689: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-10-22 20:36:41.964970: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2499995000 Hz
2020-10-22 20:36:41.965167: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x49a0ac0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-10-22 20:36:41.965183: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host

## Remote Deployment

This will deploy a remote end to test

In [5]:
!curl

curl: try 'curl --help' or 'curl --manual' for more information


In [6]:
!curl http://fairing-service-p2qsx.eksworkshop.svc.cluster.local:5000/predict
        

curl: (56) Recv failure: Connection reset by peer
